# Read Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_balanced = pd.read_pickle('train_balanced.pkl')
val_balanced = pd.read_pickle('val_balanced.pkl')
test_balanced = pd.read_pickle('test_balanced.pkl')
VOCAB_BALANCED_SIZE = set()
for tokenized in train_balanced.tokenized:
    for token in tokenized:
        VOCAB_BALANCED_SIZE.add(token)
max(VOCAB_BALANCED_SIZE)     

7130

In [3]:
train_unbalanced = pd.read_pickle('train_unbalanced.pkl')
val_unbalanced = pd.read_pickle('val_unbalanced.pkl')
test_unbalanced = pd.read_pickle('test_unbalanced.pkl')

VOCAB_UNBALANCED_SIZE = set()
for tokenized in train_unbalanced.tokenized:
    for token in tokenized:
        VOCAB_UNBALANCED_SIZE.add(token)

max(VOCAB_UNBALANCED_SIZE)

6767

In [4]:
len(train_balanced), len(train_unbalanced)

(2546, 1906)

In [5]:
from torch.utils.data import Dataset, DataLoader
class ClassificationDataset(Dataset):
    def __init__(self, data, X, y, is_embeddings):
        self.X = data[X]
        self.y = data[y]
        self.is_embeddings = is_embeddings
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        if self.is_embeddings:
            return torch.tensor(self.X[idx], dtype=torch.float), torch.tensor(self.y[idx], dtype=torch.long)
        return torch.tensor(self.X[idx], dtype=torch.int), torch.tensor(self.y[idx], dtype=torch.long)


In [6]:
train_dataset_embed_balanced = ClassificationDataset(train_balanced, "precomputed_embeddings", "class", True)
val_dataset_embed_balanced = ClassificationDataset(val_balanced, "precomputed_embeddings", "class", True)
test_dataset_embed_balanced = ClassificationDataset(test_balanced, "precomputed_embeddings", "class", True)

train_loader_embed_balanced = DataLoader(train_dataset_embed_balanced, batch_size=8, shuffle=True)
val_loader_embed_balanced = DataLoader(val_dataset_embed_balanced, batch_size=8, shuffle=False)
test_loader_embed_balanced = DataLoader(test_dataset_embed_balanced, batch_size=8, shuffle=False)

In [7]:
train_dataset_balanced = ClassificationDataset(train_balanced, "tokenized", "class", False)
val_dataset_balanced = ClassificationDataset(val_balanced, "tokenized", "class", False)
test_dataset_balanced = ClassificationDataset(test_balanced, "tokenized", "class", False)

train_loader_balanced = DataLoader(train_dataset_balanced, batch_size=16, shuffle=True)
val_loader_balanced = DataLoader(val_dataset_balanced, batch_size=16, shuffle=False)
test_loader_balanced = DataLoader(test_dataset_balanced, batch_size=16, shuffle=False)

In [8]:
train_dataset_embed_unbalanced = ClassificationDataset(train_unbalanced, "precomputed_embeddings", "class", True)
val_dataset_embed_unbalanced = ClassificationDataset(val_unbalanced, "precomputed_embeddings", "class", True)
test_dataset_embed_unbalanced = ClassificationDataset(test_unbalanced, "precomputed_embeddings", "class", True)

train_loader_embed_unbalanced = DataLoader(train_dataset_embed_unbalanced, batch_size=8, shuffle=True)
val_loader_embed_unbalanced = DataLoader(val_dataset_embed_unbalanced, batch_size=8, shuffle=False)
test_loader_embed_unbalanced = DataLoader(test_dataset_embed_unbalanced, batch_size=8, shuffle=False)

In [9]:
train_dataset_unbalanced = ClassificationDataset(train_unbalanced, "tokenized", "class", False)
val_dataset_unbalanced = ClassificationDataset(val_unbalanced, "tokenized", "class", False)
test_dataset_unbalanced = ClassificationDataset(test_unbalanced, "tokenized", "class", False)

train_loader_unbalanced = DataLoader(train_dataset_unbalanced, batch_size=16, shuffle=True)
val_loader_unbalanced = DataLoader(val_dataset_unbalanced, batch_size=16, shuffle=False)
test_loader_unbalanced = DataLoader(test_dataset_unbalanced, batch_size=16, shuffle=False)

# RNN

In [14]:
from torch import nn
from torch.optim import Adam
from torch.functional import F

In [15]:
class LSTM(nn.Module):
    def __init__(self,
              vocab_size,
              embedding_dim,
              hidden_dim,
              num_layers,
              num_classes,
              max_len,
              bidirectional,
              dropout=0,
              is_embedding_layer = True,
    ):
        super().__init__()
        self.is_embedding_layer = is_embedding_layer
        # Embeddings, which can be pretrained or normally trained
        if (self.is_embedding_layer):
            self.embeddings = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=embedding_dim
            )
        # LSTM Layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, bidirectional=bidirectional, dropout=dropout)
        # Linear Layer
        if bidirectional:
            self.linear = nn.Linear(max_len * hidden_dim * 2, num_classes)
        else:
            self.linear = nn.Linear(max_len * hidden_dim, num_classes)
       
        self.softmax = nn.Softmax()
    def forward(self, x):
        if self.is_embedding_layer:
            x = self.embeddings(x)
        lstm_out, _ = self.lstm(x)
        if self.is_embedding_layer:
            lstm_out = lstm_out.reshape(lstm_out.shape[0], -1)
        linear = self.linear(lstm_out)
        return linear

# Train & Validate 

In [16]:
import gc
import torch
import optuna
import itertools
from tqdm import tqdm
from sklearn.metrics import f1_score

In [17]:
NUM_CLASSES=4
MAX_LEN=52
def objective_balanced(trial, epochs=3):
    # Hyperparameter search space
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    embedding_dim = trial.suggest_categorical("embedding_dim", [128, 256, 512])
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256, 512])
    num_layers = trial.suggest_int("num_layers", 1, 4)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    model = LSTM(
        vocab_size=len(VOCAB_BALANCED_SIZE),
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=NUM_CLASSES,
        max_len=MAX_LEN,
        bidirectional=bidirectional,
        is_embedding_layer=True
    ).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)
    
	
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader_balanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()

            # Forward pass
            y_preds = model(X_batch)
            loss = criterion(y_preds, y_batch)

            # Backward pass
            loss.backward()
            optimizer.step()
            

    # Validation
    model.eval()
    y_preds_list = []
    y_true_list = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader_balanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            y_preds = model(X_batch)

            # Move predictions to CPU to avoid VRAM overload
            y_preds_cpu = y_preds.detach().cpu().numpy()
            y_batch_cpu =y_batch.detach().cpu().numpy()
            

            # Append predictions and true labels to lists for F1 computation
            y_preds_list.extend(np.argmax(y_preds_cpu, axis=1))
            y_true_list.extend(y_batch_cpu)

            # Clear memory after each batch
            del X_batch, y_batch, y_preds
            torch.cuda.empty_cache()
            gc.collect()
    # Compute F1 score incrementally
    f1 = f1_score(np.array(y_true_list), np.array(y_preds_list), average="weighted", labels=[0, 1, 2, 3])
    
    # Cleanup model and cache after evaluation
    del model
    torch.cuda.empty_cache()
    gc.collect()

    return f1


In [23]:
# ---- Run the Optuna Study ----
study_balanced = optuna.create_study(direction="maximize")
study_balanced.optimize(objective_balanced, n_trials=15)

print("Best hyperparameters:", study_balanced.best_trial.params)

[I 2025-05-10 21:20:57,687] A new study created in memory with name: no-name-970ea8b1-3e1d-4abf-b018-518bc10bbcb5
[I 2025-05-10 21:21:03,490] Trial 0 finished with value: 0.49562191721436255 and parameters: {'embedding_dim': 512, 'hidden_dim': 64, 'num_layers': 4, 'bidirectional': True, 'lr': 0.0012503183104253229}. Best is trial 0 with value: 0.49562191721436255.
[I 2025-05-10 21:21:08,594] Trial 1 finished with value: 0.5593249409231319 and parameters: {'embedding_dim': 128, 'hidden_dim': 64, 'num_layers': 3, 'bidirectional': True, 'lr': 0.00213009620150366}. Best is trial 1 with value: 0.5593249409231319.
[I 2025-05-10 21:21:13,238] Trial 2 finished with value: 0.5894177933929033 and parameters: {'embedding_dim': 256, 'hidden_dim': 64, 'num_layers': 1, 'bidirectional': True, 'lr': 0.0016631208817564633}. Best is trial 2 with value: 0.5894177933929033.
[I 2025-05-10 21:21:18,653] Trial 3 finished with value: 0.5327775192288556 and parameters: {'embedding_dim': 512, 'hidden_dim': 128,

Best hyperparameters: {'embedding_dim': 256, 'hidden_dim': 64, 'num_layers': 1, 'bidirectional': True, 'lr': 0.0013699937842235066}


In [25]:
def objective_unbalanced(trial, epochs=3):
    # Hyperparameter search space
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    embedding_dim = trial.suggest_categorical("embedding_dim", [128, 256, 512])
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256, 512])
    num_layers = trial.suggest_int("num_layers", 1, 4)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])

    NUM_CLASSES=4
    MAX_LEN=52
    
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    model = LSTM(
        vocab_size=len(VOCAB_UNBALANCED_SIZE) + 1000,
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=NUM_CLASSES,
        max_len=MAX_LEN,
        bidirectional=bidirectional,
        is_embedding_layer=True
    ).to(DEVICE)
    weight=torch.tensor([0.9589603283173734, 1.7014563106796117, 1.874331550802139, 0.55327545382794]).to(DEVICE)
    criterion = nn.CrossEntropyLoss(weight=weight)
    optimizer = Adam(model.parameters(), lr=lr)
    
	
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader_unbalanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()

            # Forward pass
            y_preds = model(X_batch)
            loss = criterion(y_preds, y_batch)

            # Backward pass
            loss.backward()
            optimizer.step()
            

    # Validation
    model.eval()
    y_preds_list = []
    y_true_list = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader_unbalanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            y_preds = model(X_batch)

            # Move predictions to CPU to avoid VRAM overload
            y_preds_cpu = y_preds.detach().cpu().numpy()
            y_batch_cpu =y_batch.detach().cpu().numpy()
            

            # Append predictions and true labels to lists for F1 computation
            y_preds_list.extend(np.argmax(y_preds_cpu, axis=1))
            y_true_list.extend(y_batch_cpu)

            # Clear memory after each batch
            del X_batch, y_batch, y_preds
            torch.cuda.empty_cache()
            gc.collect()
    # Compute F1 score incrementally
    f1 = f1_score(np.array(y_true_list), np.array(y_preds_list), average="weighted", labels=[0, 1, 2, 3])
    
    # Cleanup model and cache after evaluation
    del model
    torch.cuda.empty_cache()
    gc.collect()

    return f1


In [26]:
# ---- Run the Optuna Study ----
study_unbalanced = optuna.create_study(direction="maximize")
study_unbalanced.optimize(objective_unbalanced, n_trials=15)

print("Best hyperparameters:", study_unbalanced.best_trial.params)

[I 2025-05-10 21:22:23,784] A new study created in memory with name: no-name-2a610629-ed94-4bbf-b5c1-0686f83f2058
[I 2025-05-10 21:22:28,773] Trial 0 finished with value: 0.20662065037234217 and parameters: {'embedding_dim': 256, 'hidden_dim': 128, 'num_layers': 4, 'bidirectional': True, 'lr': 0.0033881903497482596}. Best is trial 0 with value: 0.20662065037234217.
[I 2025-05-10 21:22:33,257] Trial 1 finished with value: 0.31648367789005893 and parameters: {'embedding_dim': 512, 'hidden_dim': 64, 'num_layers': 3, 'bidirectional': True, 'lr': 0.00109842339743824}. Best is trial 1 with value: 0.31648367789005893.
[I 2025-05-10 21:22:37,190] Trial 2 finished with value: 0.39146082803936305 and parameters: {'embedding_dim': 128, 'hidden_dim': 128, 'num_layers': 1, 'bidirectional': False, 'lr': 0.0003250948179036696}. Best is trial 2 with value: 0.39146082803936305.
[I 2025-05-10 21:22:41,446] Trial 3 finished with value: 0.326549127785771 and parameters: {'embedding_dim': 512, 'hidden_dim'

Best hyperparameters: {'embedding_dim': 128, 'hidden_dim': 64, 'num_layers': 2, 'bidirectional': False, 'lr': 0.0004960450141084295}


In [18]:
def objective_embed_balanced(trial, epochs=5):
    
    # Hyperparameter search space
    DEVICE = 'cuda'
    embedding_dim = trial.suggest_categorical("embedding_dim", [768])
    hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256, 512])
    num_layers = trial.suggest_int("num_layers", 2, 6)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    dropout = trial.suggest_categorical("dropout", [0.1, 0.25, 0.5])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    NUM_CLASSES=4
    model = LSTM(
        vocab_size=len(VOCAB_BALANCED_SIZE),
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=NUM_CLASSES,
        max_len=1,
        bidirectional=bidirectional,
        dropout=dropout,
        is_embedding_layer=False
    ).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)
    
	
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader_embed_balanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()

            # Forward pass
            y_preds = model(X_batch)
            loss = criterion(y_preds, y_batch)

            # Backward pass
            loss.backward()
            optimizer.step()
            

    # Validation
    model.eval()
    y_preds_list = []
    y_true_list = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader_embed_balanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            y_preds = model(X_batch)

            # Move predictions to CPU to avoid VRAM overload
            y_preds_cpu = y_preds.detach().cpu().numpy()
            y_batch_cpu =y_batch.detach().cpu().numpy()
            

            # Append predictions and true labels to lists for F1 computation
            y_preds_list.extend(np.argmax(y_preds_cpu, axis=1))
            y_true_list.extend(y_batch_cpu)

            # Clear memory after each batch
            del X_batch, y_batch, y_preds
            torch.cuda.empty_cache()
            gc.collect()
    # Compute F1 score incrementally
    f1 = f1_score(np.array(y_true_list), np.array(y_preds_list), average="weighted", labels=[0, 1, 2, 3])
    
    # Cleanup model and cache after evaluation
    del model
    torch.cuda.empty_cache()
    gc.collect()

    return f1



In [19]:
# ---- Run the Optuna Study ----
study_embed_balanced = optuna.create_study(direction="maximize")
study_embed_balanced.optimize(objective_embed_balanced, n_trials=30)

print("Best hyperparameters:", study_embed_balanced.best_trial.params)

[I 2025-05-10 21:08:57,643] A new study created in memory with name: no-name-7215fef3-cb1a-4778-a19b-8404d86bd29e
[I 2025-05-10 21:09:08,873] Trial 0 finished with value: 0.6656562587493309 and parameters: {'embedding_dim': 768, 'hidden_dim': 128, 'num_layers': 2, 'bidirectional': True, 'dropout': 0.1, 'lr': 0.002844323381139709}. Best is trial 0 with value: 0.6656562587493309.
[I 2025-05-10 21:09:19,715] Trial 1 finished with value: 0.7174626058109882 and parameters: {'embedding_dim': 768, 'hidden_dim': 512, 'num_layers': 2, 'bidirectional': False, 'dropout': 0.5, 'lr': 0.0014406787560723709}. Best is trial 1 with value: 0.7174626058109882.
[I 2025-05-10 21:09:33,798] Trial 2 finished with value: 0.5944742203351513 and parameters: {'embedding_dim': 768, 'hidden_dim': 128, 'num_layers': 6, 'bidirectional': True, 'dropout': 0.1, 'lr': 0.0019966743816257613}. Best is trial 1 with value: 0.7174626058109882.
[I 2025-05-10 21:09:45,554] Trial 3 finished with value: 0.6448282774905677 and pa

Best hyperparameters: {'embedding_dim': 768, 'hidden_dim': 512, 'num_layers': 2, 'bidirectional': False, 'dropout': 0.5, 'lr': 0.0001013927614287239}


In [20]:
def objective_embed_unbalanced(trial, epochs=5):
    
    # Hyperparameter search space
    DEVICE = 'cuda'
    embedding_dim = trial.suggest_categorical("embedding_dim", [768])
    hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256, 512])
    num_layers = trial.suggest_int("num_layers", 2, 6)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    dropout = trial.suggest_categorical("dropout", [0.1, 0.25, 0.5])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    NUM_CLASSES=4
    model = LSTM(
        vocab_size=len(VOCAB_UNBALANCED_SIZE)+1000,
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=NUM_CLASSES,
        max_len=1,
        bidirectional=bidirectional,
        dropout=dropout,
        is_embedding_layer=False
    ).to(DEVICE)
    weight=torch.tensor([0.9589603283173734, 1.7014563106796117, 1.874331550802139, 0.55327545382794]).to(DEVICE)
    criterion = nn.CrossEntropyLoss(weight=weight)
    optimizer = Adam(model.parameters(), lr=lr)
    
	
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader_embed_unbalanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()

            # Forward pass
            y_preds = model(X_batch)
            loss = criterion(y_preds, y_batch)

            # Backward pass
            loss.backward()
            optimizer.step()
            

    # Validation
    model.eval()
    y_preds_list = []
    y_true_list = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader_embed_unbalanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            y_preds = model(X_batch)

            # Move predictions to CPU to avoid VRAM overload
            y_preds_cpu = y_preds.detach().cpu().numpy()
            y_batch_cpu =y_batch.detach().cpu().numpy()
            

            # Append predictions and true labels to lists for F1 computation
            y_preds_list.extend(np.argmax(y_preds_cpu, axis=1))
            y_true_list.extend(y_batch_cpu)

            # Clear memory after each batch
            del X_batch, y_batch, y_preds
            torch.cuda.empty_cache()
            gc.collect()
    # Compute F1 score incrementally
    f1 = f1_score(np.array(y_true_list), np.array(y_preds_list), average="weighted", labels=[0, 1, 2, 3])
    
    # Cleanup model and cache after evaluation
    del model
    torch.cuda.empty_cache()
    gc.collect()

    return f1



In [21]:
# ---- Run the Optuna Study ----
study_embed_unbalanced = optuna.create_study(direction="maximize")
study_embed_unbalanced.optimize(objective_embed_unbalanced, n_trials=30)

print("Best hyperparameters:", study_embed_unbalanced.best_trial.params)

[I 2025-05-10 21:15:10,199] A new study created in memory with name: no-name-d1439657-0221-43cf-a542-6adace99596a
[I 2025-05-10 21:15:20,840] Trial 0 finished with value: 0.6170648256282029 and parameters: {'embedding_dim': 768, 'hidden_dim': 256, 'num_layers': 4, 'bidirectional': False, 'dropout': 0.5, 'lr': 0.00386706593804655}. Best is trial 0 with value: 0.6170648256282029.
[I 2025-05-10 21:15:43,674] Trial 1 finished with value: 0.5576541145300451 and parameters: {'embedding_dim': 768, 'hidden_dim': 512, 'num_layers': 6, 'bidirectional': True, 'dropout': 0.5, 'lr': 0.001280537310804523}. Best is trial 0 with value: 0.6170648256282029.
[I 2025-05-10 21:15:54,972] Trial 2 finished with value: 0.5412348568537367 and parameters: {'embedding_dim': 768, 'hidden_dim': 256, 'num_layers': 5, 'bidirectional': False, 'dropout': 0.5, 'lr': 0.003151375858270054}. Best is trial 0 with value: 0.6170648256282029.
[I 2025-05-10 21:16:04,455] Trial 3 finished with value: 0.6952140623543489 and para

Best hyperparameters: {'embedding_dim': 768, 'hidden_dim': 512, 'num_layers': 2, 'bidirectional': False, 'dropout': 0.25, 'lr': 0.00046722568383779746}


In [65]:
def train_val(
        model: LSTM,
        optim: torch.optim.Optimizer,
        criterion: nn.CrossEntropyLoss,
        epochs: int,
        train_dataloader: DataLoader,
        val_dataloader: DataLoader,
        device
    ):
    best_f1 = 0
    best_model = None
    model.to(device)

    for epoch in tqdm(range(epochs)):
        model.train()
        train_loss = 0
        y_preds_train = []
        y_true_train = []

        for train_X, train_y in train_dataloader:
            train_X, train_y = train_X.to(device), train_y.to(device).long()

            y_preds = model(train_X)
            loss = criterion(y_preds, train_y)

            optim.zero_grad()
            loss.backward()
            optim.step()

            train_loss += loss.item()
            y_preds_train.extend(torch.argmax(y_preds, dim=1).cpu().numpy())
            y_true_train.extend(train_y.cpu().numpy())

            del train_X, train_y, y_preds
            torch.cuda.empty_cache()
            gc.collect()

        train_f1 = f1_score(y_true_train, y_preds_train, average="weighted", labels=[0, 1, 2, 3])

        # Validation
        model.eval()
        val_loss = 0
        y_preds_val = []
        y_true_val = []

        with torch.no_grad():
            for val_X, val_y in val_dataloader:
                val_X, val_y = val_X.to(device), val_y.to(device).long()

                y_preds = model(val_X)
                loss = criterion(y_preds, val_y)
                val_loss += loss.item()

                y_preds_val.extend(torch.argmax(y_preds, dim=1).cpu().numpy())
                y_true_val.extend(val_y.cpu().numpy())

                del val_X, val_y, y_preds
                torch.cuda.empty_cache()
                gc.collect()

        val_f1 = f1_score(y_true_val, y_preds_val, average="weighted", labels=[0, 1, 2, 3])

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model = model.state_dict()

        print(
            f"Epoch {epoch+1}/{epochs}, "
            f"Train Loss: {train_loss/len(train_dataloader):.4f}, "
            f"Val Loss: {val_loss/len(val_dataloader):.4f}, "
            f"Train F1: {train_f1*100:.2f}%, "
            f"Val F1: {val_f1*100:.2f}%"
        )
    model.load_state_dict(best_model)
    return model


In [66]:
DEVICE='cuda'

In [67]:
model_balanced = LSTM(
			vocab_size=len(VOCAB_BALANCED_SIZE),
			embedding_dim=study_balanced.best_trial.params['embedding_dim'],
			hidden_dim=study_balanced.best_trial.params['hidden_dim'],
			num_layers=study_balanced.best_trial.params['num_layers'],
			num_classes=NUM_CLASSES,
			max_len=MAX_LEN,
			bidirectional=study_balanced.best_trial.params['bidirectional'],
            is_embedding_layer=True
		)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model_balanced.parameters(), lr=study_balanced.best_trial.params['lr'])
best_balanced = train_val(
    model_balanced,
    optimizer,
    criterion,
    epochs = 5,
    train_dataloader = train_loader_balanced,
    val_dataloader = val_loader_balanced,
    device=DEVICE
)

 20%|█████████████████                                                                    | 1/5 [00:27<01:50, 27.62s/it]

Epoch 1/5, Train Loss: 1.2460, Val Loss: 1.1144, Train F1: 37.92%, Val F1: 48.08%


 40%|██████████████████████████████████                                                   | 2/5 [00:55<01:23, 27.93s/it]

Epoch 2/5, Train Loss: 0.8498, Val Loss: 1.0266, Train F1: 66.84%, Val F1: 62.87%


 60%|███████████████████████████████████████████████████                                  | 3/5 [01:24<00:56, 28.26s/it]

Epoch 3/5, Train Loss: 0.5234, Val Loss: 1.0473, Train F1: 82.26%, Val F1: 65.12%


 80%|████████████████████████████████████████████████████████████████████                 | 4/5 [01:52<00:28, 28.29s/it]

Epoch 4/5, Train Loss: 0.3523, Val Loss: 1.1165, Train F1: 87.80%, Val F1: 64.57%


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:21<00:00, 28.22s/it]

Epoch 5/5, Train Loss: 0.2642, Val Loss: 1.2113, Train F1: 90.36%, Val F1: 63.78%


In [76]:
model_embed_balanced = LSTM(
			vocab_size=len(VOCAB_BALANCED_SIZE),
			embedding_dim=study_embed_balanced.best_trial.params['embedding_dim'],
			hidden_dim=study_embed_balanced.best_trial.params['hidden_dim'],
			num_layers=study_embed_balanced.best_trial.params['num_layers'],
			num_classes=NUM_CLASSES,
			max_len=1,
			bidirectional=study_embed_balanced.best_trial.params['bidirectional'],
            is_embedding_layer=False
		)
criterion = nn.CrossEntropyLoss()
optimizer_embed_balanced = Adam(model_embed_balanced.parameters(), lr=study_embed_balanced.best_trial.params['lr'])

best_embed_balanced=train_val(
    model_embed_balanced,
    optimizer_embed_balanced,
    criterion,
    epochs = 5,
    train_dataloader = train_loader_embed_balanced,
    val_dataloader = val_loader_embed_balanced,
    device=DEVICE
)

 20%|█████████████████                                                                    | 1/5 [00:56<03:44, 56.14s/it]

Epoch 1/5, Train Loss: 1.1308, Val Loss: 0.8757, Train F1: 48.37%, Val F1: 60.85%


 40%|██████████████████████████████████                                                   | 2/5 [01:52<02:48, 56.11s/it]

Epoch 2/5, Train Loss: 0.8737, Val Loss: 0.7626, Train F1: 62.67%, Val F1: 67.98%


 60%|███████████████████████████████████████████████████                                  | 3/5 [02:48<01:52, 56.24s/it]

Epoch 3/5, Train Loss: 0.7705, Val Loss: 0.7089, Train F1: 66.66%, Val F1: 74.49%


 80%|████████████████████████████████████████████████████████████████████                 | 4/5 [04:01<01:02, 62.84s/it]

Epoch 4/5, Train Loss: 0.6963, Val Loss: 0.7124, Train F1: 70.19%, Val F1: 70.72%


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [05:02<00:00, 60.53s/it]

Epoch 5/5, Train Loss: 0.6242, Val Loss: 0.7430, Train F1: 72.81%, Val F1: 73.02%


In [77]:
model_unbalanced = LSTM(
			vocab_size=len(VOCAB_UNBALANCED_SIZE) + 1000,
			embedding_dim=study_unbalanced.best_trial.params['embedding_dim'],
			hidden_dim=study_unbalanced.best_trial.params['hidden_dim'],
			num_layers=study_unbalanced.best_trial.params['num_layers'],
			num_classes=NUM_CLASSES,
			max_len=MAX_LEN,
			bidirectional=study_unbalanced.best_trial.params['bidirectional'],
            is_embedding_layer=True
		)
weight=torch.tensor([0.9589603283173734, 1.7014563106796117, 1.874331550802139, 0.55327545382794]).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=weight)
optimizer_unbalanced = Adam(model_unbalanced.parameters(), lr=study_unbalanced.best_trial.params['lr'])
best_unbalanced = train_val(
    model_unbalanced,
    optimizer_unbalanced,
    criterion,
    epochs = 5,
    train_dataloader = train_loader_unbalanced,
    val_dataloader = val_loader_unbalanced,
    device=DEVICE
)

 20%|█████████████████                                                                    | 1/5 [00:21<01:26, 21.53s/it]

Epoch 1/5, Train Loss: 1.3904, Val Loss: 1.3893, Train F1: 29.05%, Val F1: 11.03%


 40%|██████████████████████████████████                                                   | 2/5 [00:43<01:06, 22.04s/it]

Epoch 2/5, Train Loss: 1.3798, Val Loss: 1.3929, Train F1: 32.75%, Val F1: 4.69%


 60%|███████████████████████████████████████████████████                                  | 3/5 [01:06<00:44, 22.18s/it]

Epoch 3/5, Train Loss: 1.3684, Val Loss: 1.3769, Train F1: 38.18%, Val F1: 11.15%


 80%|████████████████████████████████████████████████████████████████████                 | 4/5 [01:29<00:22, 22.52s/it]

Epoch 4/5, Train Loss: 1.3380, Val Loss: 1.3554, Train F1: 42.82%, Val F1: 35.87%


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:52<00:00, 22.50s/it]

Epoch 5/5, Train Loss: 1.2429, Val Loss: 1.3668, Train F1: 49.48%, Val F1: 30.81%


In [78]:
model_embed_unbalanced = LSTM(
			vocab_size=len(VOCAB_UNBALANCED_SIZE) + 1000,
			embedding_dim=study_embed_unbalanced.best_trial.params['embedding_dim'],
			hidden_dim=study_embed_unbalanced.best_trial.params['hidden_dim'],
			num_layers=study_embed_unbalanced.best_trial.params['num_layers'],
			num_classes=NUM_CLASSES,
			max_len=1,
			bidirectional=study_embed_unbalanced.best_trial.params['bidirectional'],
            is_embedding_layer=False
		).to(DEVICE)
weight=torch.tensor([0.9589603283173734, 1.7014563106796117, 1.874331550802139, 0.55327545382794]).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=weight)
optimizer_embed_unbalanced = Adam(model_embed_unbalanced.parameters(), lr=study_embed_unbalanced.best_trial.params['lr'])

best_embed_unbalanced=train_val(
    model_embed_unbalanced,
    optimizer_embed_unbalanced,
    criterion,
    epochs = 5,
    train_dataloader = train_loader_embed_unbalanced,
    val_dataloader = val_loader_embed_unbalanced,
    device=DEVICE
)

 20%|█████████████████                                                                    | 1/5 [00:52<03:28, 52.07s/it]

Epoch 1/5, Train Loss: 1.0066, Val Loss: 0.8611, Train F1: 61.65%, Val F1: 66.33%


 40%|██████████████████████████████████                                                   | 2/5 [01:44<02:37, 52.46s/it]

Epoch 2/5, Train Loss: 0.6678, Val Loss: 0.7779, Train F1: 75.12%, Val F1: 68.70%


 60%|███████████████████████████████████████████████████                                  | 3/5 [02:34<01:42, 51.09s/it]

Epoch 3/5, Train Loss: 0.4694, Val Loss: 0.8453, Train F1: 82.45%, Val F1: 69.33%


 80%|████████████████████████████████████████████████████████████████████                 | 4/5 [03:26<00:51, 51.43s/it]

Epoch 4/5, Train Loss: 0.2954, Val Loss: 1.0231, Train F1: 87.93%, Val F1: 68.91%


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:17<00:00, 51.47s/it]

Epoch 5/5, Train Loss: 0.1661, Val Loss: 1.3642, Train F1: 93.42%, Val F1: 69.74%


# Test

In [73]:
from sklearn.metrics import classification_report, confusion_matrix

In [74]:
def test(
    model: LSTM,
    criterion: nn.CrossEntropyLoss,
    test_dataloader: DataLoader,
    device
):
    model.eval()
    test_loss = 0
    test_correct = 0
    test_total = 0
    y_preds_list = []
    y_true_list = []

    with torch.no_grad():
        for test_X, test_y in test_dataloader:
            test_X = test_X.to(device)
            test_y = test_y.to(device).long()

            y_preds = model(test_X)
            loss = criterion(y_preds, test_y)
            test_loss += loss.item()

            predicted = torch.argmax(y_preds, dim=1)
            test_correct += (predicted == test_y).sum().item()
            test_total += test_y.size(0)

            y_preds_list.extend(predicted.detach().cpu().numpy())
            y_true_list.extend(test_y.detach().cpu().numpy())

            del test_X, test_y, y_preds
            torch.cuda.empty_cache()
            gc.collect()

    acc = 100 * test_correct / test_total
    f1 = f1_score(y_true_list, y_preds_list, average="weighted", labels=[0, 1, 2, 3])

    print(
        f"\nTest Loss: {test_loss/len(test_dataloader):.4f}, "
        f"Test Accuracy: {acc:.2f}%, "
        f"Test F1 Score: {f1:.2f}"
    )
    
    print("\nClassification Report:")
    print(classification_report(y_true_list, y_preds_list, labels=[0, 1, 2, 3]))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true_list, y_preds_list, labels=[0, 1, 2, 3]))

In [79]:
criterion=nn.CrossEntropyLoss()
test(best_balanced, criterion, test_loader_balanced, 'cuda')


Test Loss: 1.0844, Test Accuracy: 59.89%, Test F1 Score: 0.58

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.72      0.64       148
           1       0.41      0.32      0.36        82
           2       0.45      0.23      0.30        75
           3       0.69      0.73      0.71       256

    accuracy                           0.60       561
   macro avg       0.53      0.50      0.50       561
weighted avg       0.58      0.60      0.58       561


Confusion Matrix:
[[107   6   7  28]
 [ 23  26   6  27]
 [ 18  11  17  29]
 [ 41  21   8 186]]


In [80]:
test(best_embed_balanced, criterion, test_loader_embed_balanced, 'cuda')


Test Loss: 0.7371, Test Accuracy: 71.48%, Test F1 Score: 0.71

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.77      0.73       148
           1       0.57      0.61      0.59        82
           2       0.62      0.51      0.56        75
           3       0.81      0.78      0.79       256

    accuracy                           0.71       561
   macro avg       0.67      0.67      0.67       561
weighted avg       0.72      0.71      0.71       561


Confusion Matrix:
[[114   6   5  23]
 [ 15  50   3  14]
 [ 18   8  38  11]
 [ 19  23  15 199]]


In [81]:
criterion = nn.CrossEntropyLoss(weight=weight)
test(best_unbalanced, criterion, test_loader_unbalanced, 'cuda')


Test Loss: 1.3839, Test Accuracy: 28.88%, Test F1 Score: 0.30

Classification Report:
              precision    recall  f1-score   support

           0       0.34      0.24      0.28       148
           1       0.16      0.09      0.11        82
           2       0.16      0.47      0.23        75
           3       0.44      0.33      0.38       256

    accuracy                           0.29       561
   macro avg       0.28      0.28      0.25       561
weighted avg       0.34      0.29      0.30       561


Confusion Matrix:
[[ 36  13  50  49]
 [ 18   7  28  29]
 [ 10   3  35  27]
 [ 41  20 111  84]]


In [83]:
criterion = nn.CrossEntropyLoss(weight=weight)
test(best_unbalanced, criterion, test_loader_unbalanced, 'cuda')


Test Loss: 1.3839, Test Accuracy: 28.88%, Test F1 Score: 0.30

Classification Report:
              precision    recall  f1-score   support

           0       0.34      0.24      0.28       148
           1       0.16      0.09      0.11        82
           2       0.16      0.47      0.23        75
           3       0.44      0.33      0.38       256

    accuracy                           0.29       561
   macro avg       0.28      0.28      0.25       561
weighted avg       0.34      0.29      0.30       561


Confusion Matrix:
[[ 36  13  50  49]
 [ 18   7  28  29]
 [ 10   3  35  27]
 [ 41  20 111  84]]


In [82]:
criterion = nn.CrossEntropyLoss(weight=weight)
test(best_embed_unbalanced, criterion, test_loader_embed_unbalanced, 'cuda')


Test Loss: 1.1867, Test Accuracy: 69.34%, Test F1 Score: 0.69

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.74      0.70       148
           1       0.56      0.70      0.62        82
           2       0.62      0.45      0.52        75
           3       0.78      0.74      0.76       256

    accuracy                           0.69       561
   macro avg       0.66      0.66      0.65       561
weighted avg       0.70      0.69      0.69       561


Confusion Matrix:
[[109   9   6  24]
 [ 12  57   3  10]
 [ 15   7  34  19]
 [ 26  29  12 189]]
